In [1]:
import pandas as pd
import numpy as np

In [2]:
work_calendar = pd.read_csv('work_cal_incidences.csv')
work_calendar['absence_type'] = work_calendar['absence_type'].fillna('no_ausente')


In [3]:
wc2 = work_calendar.pivot_table(index='person_id',columns='absence_type', values='day', aggfunc='count', fill_value='')

In [4]:
wc2.to_csv('worker_attendance_summary.csv',index=True)